In [ ]:
import os
import glob
import pandas as pd
import networkx as nx

In [ ]:
trajectories = 'path/to/Output/cell_differentiation_maps/output_TLS-CL_2/out_k'
mapping_file = 'state_index_mapping.tsv'
mapping_dict = dict()
with open(mapping_file, 'r') as file:
    for line in file:
        line = line.strip('\n')
        number, label = line.split('\t')
        mapping_dict[int(number)] = label
print(mapping_dict)

In [ ]:
def relabel_trajector(mirrored_mapping, output_path):
    relabeled_output = output_path + 'relabeled_graphs/'
    os.makedirs(relabeled_output, exist_ok=True)

    #mirrored_mapping = {v: k for k, v in mirrored_mapping.items()}
    #print(mirrored_mapping)
    for graph_file in glob.glob(os.path.join(output_path, 'trajectories_gexf/*.gexf')):
        file_name = graph_file.split('/')[-1]
        tmp_graph = nx.read_gexf(graph_file)
        tmp_graph = nx.transitive_reduction(tmp_graph)
        label_graph = nx.DiGraph()
        for edge in tmp_graph.edges():
            a, b = edge
            a = int(a)
            b = int(b)
            if a == 0:
                label_graph.add_edge('0', mirrored_mapping[b])
            else:
                label_graph.add_edge(mirrored_mapping[a], mirrored_mapping[b])

        nodes = list(label_graph.nodes())
        if '0' not in nodes:
            for n in nodes:
                label_graph.add_edge('0', n)

        print(relabeled_output + file_name)
        nx.write_gexf(label_graph, relabeled_output + file_name)
        dot = nx.drawing.nx_pydot.to_pydot(label_graph)
        dot.write_png(relabeled_output + file_name.replace('gexf', 'png'))
    return

for i in range(2, 12):
    relabel_trajector(mapping_dict, trajectories + str(i) + '/')